如果有多个三维卷积核，那么每个核生成一个输出通道
$\begin{aligned}&\bullet\text{ 输入 }\mathbf{X}:c_i\times n_h\times n_w\\&\bullet\text{ 核 }\mathbf{W}:c_o\times c_i\times k_h\times k_w\\&\bullet\text{ 输出 }\mathbf{Y}:c_o\times m_h\hat{\times}m_w\end{aligned}$

多输入通道:对每个输入channal做卷积后再求和

In [1]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))
#d2l.corr2d就是上一节的corr2d

C:\ProgramData\anaconda3\envs\dd2l\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

多输出通道:将之前多输入通道的结果并列起来即为多输出

In [3]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [5]:
K = torch.stack((K, K + 1, K + 2), dim=0)
#torch.stack()沿着一个新维度对输入张量序列进行连接,dim新的维度
K.shape

tensor([[[[[0., 1.],
           [2., 3.]],

          [[1., 2.],
           [3., 4.]]],


         [[[1., 2.],
           [3., 4.]],

          [[2., 3.],
           [4., 5.]]],


         [[[2., 3.],
           [4., 5.]],

          [[3., 4.],
           [5., 6.]]]],



        [[[[1., 2.],
           [3., 4.]],

          [[2., 3.],
           [4., 5.]]],


         [[[2., 3.],
           [4., 5.]],

          [[3., 4.],
           [5., 6.]]],


         [[[3., 4.],
           [5., 6.]],

          [[4., 5.],
           [6., 7.]]]],



        [[[[2., 3.],
           [4., 5.]],

          [[3., 4.],
           [5., 6.]]],


         [[[3., 4.],
           [5., 6.]],

          [[4., 5.],
           [6., 7.]]],


         [[[4., 5.],
           [6., 7.]],

          [[5., 6.],
           [7., 8.]]]]])


1X1卷积

In [6]:
#通过全连接层实现1*1卷积
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)

    #先用矩阵乘法做卷积,再回复形状
    return Y.reshape((c_o, h, w))

In [8]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
#全连接做法
Y1 = corr2d_multi_in_out_1x1(X, K)
#卷积做法
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6


tensor([[[False,  True, False],
         [ True,  True,  True],
         [ True, False, False]],

        [[ True, False, False],
         [ True,  True,  True],
         [ True, False,  True]]])